In [8]:
import cv2
import mediapipe as mp
import time
import numpy as np
import os
from matplotlib import pyplot as plt

In [9]:
mp_holistic = mp.solutions.holistic  # MP holistic model
mp_drawing = mp.solutions.drawing_utils  # MP drawing utilities
mp_drawing_styles = mp.solutions.drawing_styles
pTime, cTime = 0, 0

CAMERA_INDEX = 0
CAMERA_WIDTH = 1920
CAMERA_HEIGHT = 1080
DATA_DURATION = 30
DATA_LENGTH = 30
DATA_BASE_PATH = 'MP_Dataset'
CLASS_TAKEN = 'a'


In [10]:
try:
    os.makedirs(os.path.join(DATA_BASE_PATH, CLASS_TAKEN))
except:
    pass


In [11]:
def use_holistic(image, holistic, mp_drawing):
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = holistic.process(image)

    # Draw landmark annotation on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    mp_drawing.draw_landmarks(
        image,
        results.face_landmarks,
        mp_holistic.FACEMESH_CONTOURS,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp_drawing_styles
        .get_default_face_mesh_contours_style())
    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_holistic.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles
        .get_default_pose_landmarks_style())
    mp_drawing.draw_landmarks(
        image,
        results.left_hand_landmarks,
        mp_holistic.HAND_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles
        .get_default_hand_landmarks_style())
    mp_drawing.draw_landmarks(
        image,
        results.right_hand_landmarks,
        mp_holistic.HAND_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles
        .get_default_hand_landmarks_style())
    return image, results

def show_fps(image, pTime):
    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime
    cv2.putText(image, str(int(fps)), (10, 70),
                cv2.FONT_HERSHEY_PLAIN, 3, (255, 128, 255), 3)
    return image, pTime

In [12]:
def image_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]
                    ).flatten() if results.pose_landmarks else np.zeros(132)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]
                    ).flatten() if results.face_landmarks else np.zeros(1404)
    left_hand = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]
                         ).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    right_hand = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]
                          ).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, left_hand, right_hand])


In [16]:
cap = cv2.VideoCapture(CAMERA_INDEX, cv2.CAP_DSHOW)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, CAMERA_WIDTH)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, CAMERA_HEIGHT)
with mp_holistic.Holistic(
        min_detection_confidence=0.3,
        min_tracking_confidence=0.3) as holistic:

    if not cap.isOpened():
        print("Cannot open camera")
        exit()

    frame_count = 0
    capture_frames = False
    session_count = 0
    while True:
        success, image = cap.read()
        if not success:
            print("Can't receive frame (stream end?). Exiting ...")
            break

        image, results = use_holistic(image, holistic, mp_drawing)
        image, pTime = show_fps(image, pTime)
        key = cv2.waitKey(1) & 0xFF

        # If pressed space bar create a path for new dataset
        if key == ord(' '):
            landmarks = image_keypoints(results)
            frame_count = 0
            print("Capturing frames...")

            while(os.path.exists(os.path.join(DATA_BASE_PATH, CLASS_TAKEN, str(session_count)))):
                session_count += 1

            if not os.path.exists(os.path.join(DATA_BASE_PATH, CLASS_TAKEN, str(session_count))):
                os.makedirs(os.path.join(DATA_BASE_PATH,
                            CLASS_TAKEN, str(session_count)))
                # session_count += 1

            capture_frames = True

        if capture_frames:
            cv2.imwrite(os.path.join(
                DATA_BASE_PATH, CLASS_TAKEN, str(session_count), f"frame{frame_count}.jpg"), image)
            np.save(os.path.join(
                DATA_BASE_PATH, CLASS_TAKEN, str(session_count), f"frame{frame_count}"), landmarks)
            frame_count += 1

            # Stop capturing frames after capturing 30 frames
            if frame_count == 30:
                capture_frames = False
                print("Finished capturing frames.")

        cv2.imshow("Holistic Mediapipe", image)
        if key == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


Capturing frames...
Finished capturing frames.
